In [12]:
#!pip install transformers
#!pip install datasets

^C


In [39]:
# 데이터 로드를 위함
from datasets import load_dataset

# 기본 파이썬 패키지
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

# GPT 사용을 위함
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from datasets import Dataset
# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score


In [40]:
!wget https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv


--2025-01-23 04:58:33--  https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1319001 (1.3M) [text/plain]
Saving to: ‘finance_data.csv.3’

finance_data.csv.3  100%[===================>]   1.26M  4.85MB/s    in 0.3s    

2025-01-23 04:58:34 (4.85 MB/s) - ‘finance_data.csv.3’ saved [1319001/1319001]



In [54]:
df = pd.read_csv('finance_data.csv')
print('샘플의 개수 :', len(df))

샘플의 개수 : 4846


In [55]:
df['labels'].value_counts()

labels
0    2879
1    1363
2     604
Name: count, dtype: int64

In [56]:
df['labels'] = df['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
df.head()

,labels,sentence,kor_sentence
0,0,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [57]:
df.to_csv('finance_data.csv', index=False, encoding='utf-8-sig')

In [58]:
all_data = load_dataset(
        "csv",
        data_files={
            "train": "finance_data.csv",
        },
    )


Generating train split: 0 examples [00:00, ? examples/s]

In [59]:
cs = all_data['train'].train_test_split(0.2, seed=777)
train_cs = cs["train"]
test_cs = cs["test"]


In [62]:
# 훈련 데이터를 다시 8:2로 분리 후 훈련 데이터와 검증 데이터로 저장
cs = train_cs.train_test_split(0.2, seed=777)
train_cs = cs["train"]
valid_cs = cs["test"]


In [64]:
ataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 3100
})


NameError: name 'ataset' is not defined

In [65]:
Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 776
})


NameError: name 'features' is not defined

In [63]:
Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})

NameError: name 'features' is not defined

In [ ]:
print('두번째 샘플 출력 :', train_cs['kor_sentence'][1])
print('두번째 샘플의 레이블 출력 :', train_cs['labels'][1])

In [48]:
test_sentences[:5]

NameError: name 'test_sentences' is not defined

In [49]:
from transformers import AutoModelForSequenceClassification

# 한국어 GPT 중 하나인 'skt/kogpt2-base-v2'를 사용.
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')


NameError: name 'AutoTokenizer' is not defined

In [50]:
# 최대 길이는 128
max_len = 128

def data_to_tensor (sentences, labels, max_len):
  # 정수 인코딩 과정. 각 텍스트를 토큰화한 후에 Vocabulary에 맵핑되는 정수 시퀀스로 변환한다.
  # ex) ['안녕하세요'] ==> ['안', '녕', '하세요'] ==> [231, 52, 45]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # pad_sequences는 패딩을 위한 모듈. 주어진 최대 길이를 위해서 뒤에서 패딩 토큰의 번호로 채워준다.
  # ex) [231, 52, 45] ==> [231, 52, 45, 패딩 토큰, 패딩 토큰, 패딩 토큰]
  pad_token = tokenizer.encode('<pad>')[0]
  input_ids = pad_sequences(input_ids, maxlen=max_len, value=pad_token, dtype="long", truncating="post", padding="post") 

  attention_masks = []

  for seq in input_ids:
      seq_mask = [float(i != pad_token) for i in seq]
      attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks


In [51]:
# 학습 데이터, 검증 데이터, 테스트 데이터에 대해서
# 정수 인코딩 결과, 레이블, 어텐션 마스크를 각각 inputs, labels, masks에 저장.
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels, max_len)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels, max_len)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels, max_len)


NameError: name 'train_sentences' is not defined

In [52]:
정수 인코딩 결과: tensor([ 9250,  9258, 17970,  9436, 37825,  9220,  7239,  7756, 11276, 20835,
         9133, 10644,  9427,  9706, 13872,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3])
--------------------
원본 문장 복원 결과: 그러나 그는 운전자가 2중 추돌사고를 당한 후 그를 다시 발견했다.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
--------------------
어텐션 마스크: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])
--------------------
샘플의 길이: 128
--------------------
레이블: tensor(0)


SyntaxError: invalid decimal literal (<ipython-input-52-e18863b26880>, line 15)

In [53]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


NameError: name 'train_inputs' is not defined

In [ ]:
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')


In [ ]:
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
num_labels = 3

model = AutoModelForSequenceClassification.from_pretrained("skt/kogpt2-base-v2", num_labels=num_labels)
model.cuda()


In [ ]:
# 몇 번의 에포크(전체 데이터에 대한 학습 횟수)를 할 것인지 선택
epochs = 3

# 옵티마이저 선택
optimizer = AdamW(model.parameters(), lr = 2e-5)


In [ ]:
def metrics(predictions, labels):
    # predictions: 모델이 예측한 결과값들의 리스트 또는 배열
    # labels: 실제 정답 레이블들의 리스트 또는 배열

    # 예측값과 실제 레이블을 별도의 변수에 할당
    y_pred = predictions
    y_true = labels

    # 사용 가능한 메트릭들을 계산

    # 정확도 (Accuracy)
    # 전체 예측 중에서 올바르게 예측한 비율
    accuracy = accuracy_score(y_true, y_pred)

    # 매크로 평균 F1 점수 (Macro-averaged F1 Score)
    # 클래스별로 F1 점수를 계산한 후, 그 평균을 구함
    # zero_division=0 옵션은 분모가 0일 경우 0을 반환하도록 설정
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)

    # 마이크로 평균 F1 점수 (Micro-averaged F1 Score)
    # 전체 데이터에 대해 단일 F1 점수를 계산
    # 클래스 불균형이 심한 경우에 적합
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)

    # 가중 평균 F1 점수 (Weighted-averaged F1 Score)
    # 각 클래스의 F1 점수에 해당 클래스의 샘플 수를 가중치로 곱한 후 평균을 구함
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)

    # 계산된 메트릭 결과를 딕셔너리 형태로 리턴
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average}
    return metrics


In [ ]:
def train_epoch(model, train_dataloader, optimizer, device):
    """
    하나의 에포크 동안 모델을 학습시키는 함수입니다.

    Parameters:
    model (torch.nn.Module): 학습시킬 모델 객체.
    train_dataloader (torch.utils.data.DataLoader): 학습 데이터셋의 DataLoader.
    optimizer (torch.optim.Optimizer): 최적화 알고리즘을 구현하는 객체.
    device (torch.device): 학습에 사용할 장치(CPU 또는 CUDA).

    Returns:
    float: 평균 학습 손실값.
    """

    total_train_loss = 0  # 학습 손실을 누적할 변수 초기화
    model.train()  # 모델을 학습 모드로 설정

    # 학습 데이터로더를 순회하며 배치 단위로 학습
    for step, batch in tqdm(enumerate(train_dataloader), desc="Training Batch"):
        batch = tuple(t.to(device) for t in batch)  # DataLoader에서 배치를 받아 각 텐서를 지정된 장치로 이동
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 입력 ID, 마스크, 라벨 추출

        # 모델에 배치를 전달하여 손실값 계산
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        # 손실값 추출
        loss = outputs.loss

        optimizer.zero_grad()  # 기울기(gradient) 초기화
        loss.backward()  # 역전파를 통해 기울기(gradient) 계산
        optimizer.step()  # 매개변수 업데이트

        total_train_loss += loss.item()  # 총 손실에 더함

    avg_train_loss = total_train_loss / len(train_dataloader)  # 평균 학습 손실 계산

    return avg_train_loss  # 평균 학습 손실 반환


In [ ]:
def evaluate(model, validation_dataloader, device):
    """
    모델을 사용하여 검증 데이터셋에 대한 평가를 수행하는 함수입니다.

    Parameters:
    model (torch.nn.Module): 평가할 모델 객체.
    validation_dataloader (torch.utils.data.DataLoader): 검증 데이터셋의 DataLoader.
    device (torch.device): 평가에 사용할 장치(CPU 또는 CUDA).

    Returns:
    float: 평균 검증 손실값.
    dict: 다양한 평가 지표(metrics)에 대한 값들을 담은 사전.
    """

    model.eval()  # 모델을 평가 모드로 설정

    total_eval_loss = 0  # 검증 손실을 누적할 변수 초기화
    predictions, true_labels = [], []  # 예측값과 실제 라벨값을 저장할 리스트 초기화

    # 검증 데이터로더를 순회하며 배치 단위로 평가
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)  # 배치 데이터를 디바이스로 이동
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 입력 ID, 마스크, 라벨 추출

        with torch.no_grad():  # 기울기(gradient) 계산을 수행하지 않음
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        # 모델 출력에서 손실값 추출
        if outputs.loss is not None:
            loss = outputs.loss
            total_eval_loss += loss.item()  # 총 손실에 더함

        logits = outputs.logits.detach().cpu().numpy()  # 모델 예측값(로짓)을 numpy 배열로 변환
        label_ids = b_labels.to('cpu').numpy()  # 실제 라벨값을 numpy 배열로 변환

        # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정 (예시: logits = [3.513, -0.309, -2.111] ==> 예측: 0)
        predictions.extend(np.argmax(logits, axis=1).flatten()) # 예측된 클래스를 리스트에 추가
        true_labels.extend(label_ids.flatten()) # 실제 레이블 값을 리스트에 추가

    eval_metrics = metrics(predictions, true_labels)

    return total_eval_loss / len(validation_dataloader), eval_metrics


In [ ]:
# 최소 검증 손실 초기화
min_val_loss = float('inf')

# 메인 학습 & 평가 루프
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))

    # 학습 단계
    train_epoch(model, train_dataloader, optimizer, device)

    print("\nRunning Validation...")
    # 검증 단계
    avg_val_loss, eval_metrics = evaluate(model, validation_dataloader, device)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
    print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
    print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
    print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))

    # 검증 손실이 현재까지의 최소값보다 작은 경우 체크포인트 저장
    if avg_val_loss < min_val_loss:
        print(f"Validation loss decreased ({min_val_loss:.2f} --> {avg_val_loss:.2f}).  Saving model ...")
        # 베스트 모델 저장
        torch.save(model.state_dict(), 'model_checkpoint.pt')
        # 최소 검증 손실 업데이트
        min_val_loss = avg_val_loss


In [ ]:
======== Epoch 1 / 3 ========
Training Batch: 97it [00:34,  2.81it/s]

Running Validation...
  Validation Loss: 0.43
  Accuracy: 0.84
  F1 Macro: 0.82
  F1 Micro: 0.84
  F1 Weighted: 0.84
Validation loss decreased (inf --> 0.43).  Saving model ...
======== Epoch 2 / 3 ========
Training Batch: 97it [00:33,  2.86it/s]

Running Validation...
  Validation Loss: 0.53
  Accuracy: 0.79
  F1 Macro: 0.78
  F1 Micro: 0.79
  F1 Weighted: 0.79
======== Epoch 3 / 3 ========
Training Batch: 97it [00:34,  2.84it/s]

Running Validation...
  Validation Loss: 0.55
  Accuracy: 0.83
  F1 Macro: 0.81
  F1 Micro: 0.83
  F1 Weighted: 0.83


In [ ]:
# 베스트 모델 로드
model.load_state_dict(torch.load("model_checkpoint.pt"))

avg_val_loss, eval_metrics = evaluate(model, test_dataloader, device)
print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))


In [ ]:
  Test Loss: 0.45
  Accuracy: 0.82
  F1 Macro: 0.79
  F1 Micro: 0.82
  F1 Weighted: 0.82


In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512, return_all_scores=True, function_to_apply='softmax')


In [ ]:
result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)


In [ ]:
[[{'label': 'LABEL_0', 'score': 0.018421756103634834}, {'label': 'LABEL_1', 'score': 0.9810470938682556}, {'label': 'LABEL_2', 'score': 0.0005311687709763646}]]


In [ ]:
# return_all_scores 제거
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512, function_to_apply='softmax')

result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)


In [ ]:
[{'label': 'LABEL_1', 'score': 0.9810470938682556}]


In [ ]:
label_dict = {'LABEL_0' : '중립', 'LABEL_1' : '긍정', 'LABEL_2' : '부정'}

def prediction(text):
  result = pipe(text)

  return [label_dict[result[0]['label']]]


In [ ]:
prediction('네이버가 매출이 급성장하였다')


In [ ]:
prediction('ChatGPT의 등장으로 인공지능 스타트업들은 위기다')


In [ ]:
prediction('인공지능 기술의 발전으로 누군가는 기회를 얻을 것이고, 누군가는 얻지 못할 것이다')
